In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests_html import AsyncHTMLSession
import asyncio
import nest_asyncio
import os


In [2]:
base_url = "https://data.orc.org/public/Wpub.dll"
base_country_url = "https://data.orc.org/public/WPub.dll?action=activecerts&CountryId=" 
base_boat_url = "https://data.orc.org" 

### getting names of countries

In [12]:
response = requests.get(base_url)
country_list = []

soup = BeautifulSoup(response.text, 'html.parser')
for countryId in soup.find_all('countryid'):
    country_list.append(countryId.text)

print(country_list)


['ARG', 'ARG', 'AUS', 'AUS', 'AUT', 'AUT', 'BRA', 'BRA', 'BUL', 'BUL', 'CAN', 'CAN', 'CRO', 'CRO', 'CYP', 'CYP', 'DEN', 'DEN', 'ECU', 'ECU', 'ESP', 'ESP', 'EST', 'EST', 'FIN', 'FIN', 'FRA', 'FRA', 'GBR', 'GBR', 'GER', 'GER', 'GRE', 'GRE', 'HKG', 'HKG', 'HUN', 'HUN', 'ISR', 'ISR', 'ITA', 'ITA', 'JPN', 'JPN', 'KOR', 'KOR', 'LAT', 'LAT', 'LTU', 'LTU', 'MLT', 'MLT', 'MNE', 'MNE', 'NED', 'NED', 'NOR', 'NOR', 'PER', 'PER', 'POL', 'POL', 'POR', 'POR', 'ROU', 'ROU', 'RSA', 'RSA', 'SLO', 'SLO', 'SUI', 'SUI', 'SWE', 'SWE', 'TUR', 'TUR', 'USA', 'USA']


/home/kuba/miniconda3/lib/python3.11/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


### createng function for process contry 

In [4]:
nest_asyncio.apply()
async def get_ref_links(url):
    session = AsyncHTMLSession()
    response = await session.get(url)
    await response.html.arender()

    links = []
    refno_elements = response.html.find('a[href*="/public/WPub.dll/CC/"]')
    for elem in refno_elements:
        links.append(elem.attrs['href'])

    return links

def process_country(country_name):
    country_url = base_country_url + country_name
    links = asyncio.run(get_ref_links(country_url))
    return links

### process one boat

In [5]:
def clean_text(text):
    return text.replace('\xa0', ' ').replace('Â', '').strip()

def process_boat(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    data = {}

    # Ekstrakcja sekcji BOAT
    boat_section = soup.find('span', string="BOAT").find_next('div', class_='tabular2')
    boat_labels = boat_section.find_all('span', class_='label')
    boat_data = boat_section.find_all('span', class_='data')
    
    boat_info = {}
    for label, value in zip(boat_labels, boat_data):
        boat_info[clean_text(label.text)] = clean_text(value.text)
    
    data.update(boat_info)

    # Ekstrakcja sekcji HULL
    hull_section = soup.find('span', string="HULL").find_next('div', class_='tabular2')
    hull_labels = hull_section.find_all('span', class_='label')
    hull_data = hull_section.find_all('span', class_='data')

    hull_info = {}
    for label, value in zip(hull_labels, hull_data):
        hull_info[clean_text(label.text)] = clean_text(value.text)

    data.update(hull_info)
    
    #Ekstrakcja sekcji sail areas
    sail_info = {} 
    sail_section = soup.find('span', string="SAIL AREAS").find_next('div', class_='tabular3')
    
    for tag in sail_section:
        if 'label' in tag.get('class', []):
            current_label = tag.get_text(strip=True)
        elif 'data' in tag.get('class', []):
            if current_label:
                if 'rated' in tag.get('class', []):
                    key = f"{current_label} rated"
                else:
                    key = f"{current_label} measured"
                sail_info[key] = tag.get_text(strip=True)
    
    
    data.update(sail_info)
    
    # ekstrakcja sekcji RIG
    rig_info = {}
    rig_section = soup.find('span', string="RIG").find_next('div', class_='datacol first')
    for tag in rig_section:
        if 'label' in tag.get('class', []):
            current_label = tag.get_text(strip=True)
        else:
            rig_info[current_label] = tag.get_text(strip=True)
            
    rig_section = rig_section.find_next('div', class_='datacol')
    for tag in rig_section:
        if 'label' in tag.get('class', []):
            current_label = tag.get_text(strip=True)
        else:
            rig_info[current_label] = tag.get_text(strip=True)
            
    rig_section = rig_section.find_next('div', class_='datacol')
    for tag in rig_section:
        if 'label' in tag.get('class', []):
            current_label = tag.get_text(strip=True)
        else:
            rig_info[current_label] = tag.get_text(strip=True)
    
            
    data.update(rig_info)
    
    # ekstrakcja sekcji speed
    speed_info = {} 
    table = soup.find('table', class_='boatspeeds')
    headers = [header.get_text(strip=True) for header in table.find_all('tr')[0].find_all('td')]
    
    rows = table.find_all('tr')[1:]
    for row in rows:
        cells = row.find_all('td')
        row_header = cells[0].get_text(strip=True)
        row_data = [cell.get_text(strip=True) for cell in cells[1:]]
        speed_info[row_header] = dict(zip(headers[1:], row_data))
    
    data.update(speed_info)
    
    return data

In [6]:
links = process_country("GBR")
data_dict = process_boat(base_boat_url + links[0])
data_dict


{'Class': 'FIRST 45',
 'Designer': 'BRIAND',
 'Builder': 'BENETEAU',
 'Age date': '07/2011',
 'Series date': '07/2008',
 'Offset file': 'F_45MK.OFF',
 'Data file': 'GBR2784L',
 'Length Overall': '13.670 m',
 'Maximum Beam': '4.202 m',
 'Draft': '2.377 m',
 'Displacement': '10,660 kg',
 'DLR': '6.3886',
 'IMS Division': 'Cruiser/Racer',
 'Dynamic Allowance': '0.180%',
 'Age Allowance': '0.487%',
 'Mainsail measured': '59.43',
 'Mainsail rated': '60.38',
 'Headsail Luffed measured': '55.68',
 'Headsail Luffed rated': '55.68',
 'Symmetric measured': '162.90',
 'Symmetric rated': '162.90',
 'Asymmetric measured': '175.56',
 'Asymmetric rated': '175.56',
 'Forestay tension': 'Aft',
 'Inner stay': 'None Fitted',
 'Mast material': 'Aluminum',
 'Single headsail furler': 'No',
 'Mainsail furler': 'No',
 'Articulated bowsprit': 'No',
 'Non-circular rigging': 'No',
 'Fiber rigging': 'No',
 'Adjustable mast foot': 'No',
 'Runners/Checkstays': '0',
 'Spreaders': '3',
 'Rotating mast': 'No',
 'P': '

### making database

In [7]:
def flatten_dict(d):
    flattened = {}
    for key, value in d.items():
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                flattened[f"{key} {sub_key}"] = sub_value
        else:
            flattened[key] = value
    return flattened

In [11]:
for country in country_list:
    if os.path.exists(f'output/{country}.xlsx') or country == 'GER':
        continue
        
    i = 0
    links = process_country(country)
    df = pd.DataFrame()
    for link in links:
        data_dict = process_boat(base_boat_url + link)
        flattened_dict = flatten_dict(data_dict)
        df = pd.concat([df, pd.DataFrame([flattened_dict])], sort=False, ignore_index=True)
        i += 1
        print(country, i)
        
    df.to_excel(f'output/{country}.xlsx', index=False)
    

MU_ 1
MU_ 2
MU_ 3
MU_ 4
MU_ 5
MU_ 6
MU_ 7
